# Seasonal Trading Plan Project
### This project seeks to figure out which stocks consistently go up over 30, 60 or 90 day periods, year after year, at least 80% of the time. It turns out there are many of them.

In this notebook, I created a process to download full historical EOD price data on each of the S&P 500 index components and analyze the historical patterns to find situations where seasonal trends can be taken advantage of over the course of a year.

For each stock, I calculated the following statistics:

Holding Period:  The number of days held in each rolling period - 30, 60 and 90 days each.

% Up Rows:  The percentage of rolling periods where, each year, the stock went up at least 80% of the time.

Avg Up Return:  The average return for each rolling period where the return was positive.

Avg Up StDev: The standard deviation of return for each rolling period where the return was positive. This means that the expected return should be within this +/- range from the average 67% of the time.

% Downside:  The expected average return minus the standard deviation. This means that you should at least exceed this return 67% of the time. This could provide guidance for setting stop-loss levels.

Least Pain Pt:  The time interval that offers the best case 67% downside within the sweet spot period. This is used to pinpoint the lowest risk time frame.

Best Begin Date:  The recommended entry date for best risk/reward scenario.

Best End Date:  The recommended exit date for best risk/reward scenario.

Max Consec 80%+:  The longest consecutive number of rolling periods above 80% up years. You want to see more that a few consecutive up years over the threshold to provide assurance that the trend is robust.

Total Years:  The number of years of historical data available for each stock. I filtered for stocks with 10+ years of history.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf
from pandas_datareader import data as pdr

In [2]:
# This is the path to where I store this notebook and downloaded files. You need to change this to your convenient
# spot on your own hard drive.

my_path = '/Users/bnsheehy/Documents/Investments/Seasonal_Analyses/Python'
threshold = 0.80

In [3]:
file_sp500_tickers = my_path + "/SP500_TickerList.csv"

In [4]:
# Upload a list of the S&P 500 components downloaded from Yahoo.

df_sp500_tickers = pd.read_csv (file_sp500_tickers)
df_sp500_tickers.head()

,Symbol,Company Name,GICS Sector,GICS Sub Industry
0,A,Agilent Technologies Inc,Health Care,Health Care Equipment
1,AAL,American Airlines Group,Industrials,Airlines
2,AAP,Advance Auto Parts,Consumer Discretionary,Automotive Retail
3,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals"
4,ABBV,Rickers,Health Care,Pharmaceuticals


In [5]:
# This module loops through the S&P 500 tickers, downloads the data from Yahoo and creates a separate CSV 
# file of historical data for each ticker (e.g. AAPL.csv).
# Skip this routine if you already have the CSV files available.

for index, ticker in df_sp500_tickers.iterrows():
    my_ticker = ticker['Symbol']
    
    yf_ticker = yf.Ticker(my_ticker)
    data = yf_ticker.history(period="max")
    df = pd.DataFrame(data)
    df.reset_index(level=0, inplace=True)
    df['Symbol'] = my_ticker
    df = df[['Symbol','Date','Close']]
    df.to_csv(path_or_buf = my_path + "/data/" + my_ticker +".csv", index=False)
    

- FIS: Error occurred while retrieving timeseries from Redis, keys: [RedisKey [key=FIS, cluster=finance]]


In [5]:
# Creates the dataframe container for the stats data.

df_tradelist = pd.DataFrame(index=[], columns=['my_ticker', 'hold_per', 'pct_uprows', 'max_up_return', 'min_up_return', 'avg_up_return', 'stdev_up_return', 'pct_downside', 'least_pain_pt', 'total_years', 'max_consec_beat', 'best_buy_date', 'best_sell_date'])

df_tradelist.head()

,my_ticker,hold_per,pct_uprows,max_up_return,min_up_return,avg_up_return,stdev_up_return,pct_downside,least_pain_pt,total_years,max_consec_beat,best_buy_date,best_sell_date


In [6]:
# This module grabs each ticker file, transforms it and calculates the statistics needed for a 90 day holding period.

def calc_3month_returns():
    
    global df_tradelist
    global threshold
    hold_per="3 Mos"
    
    # Convert prices to 3 month returns based on 20 trading days per month.
    dfr = df.pct_change(periods=60)
    dfr.reset_index(level=0, inplace=True)
    dfr.rename(columns={'Close':'Returns'}, inplace=True)
    dfr = dfr.round(4)
    
    # Separate out the date column into separate month, year and day values.
    dfr['Month'] = pd.DatetimeIndex(dfr['Date']).month
    dfr['Day'] = pd.DatetimeIndex(dfr['Date']).day
    dfr['Year'] = pd.DatetimeIndex(dfr['Date']).year
    dfr['M-D'] = dfr['Month'].astype(str)+'-'+dfr['Day'].astype(str)
    pd.set_option('display.max_rows', len(dfr))

    # Pivot the table to show years across the top and Month-Day values in the first column on the left.
    dfr_pivot = dfr.pivot(index='M-D', columns='Year', values='Returns')
    dfr_pivot.reset_index(level=0, inplace=True)
    dfr_pivot = pd.DataFrame(dfr_pivot)
    dfr_pivot.columns.name="Index"

    # The pivot operation created empty cells for weekends and holiday, so I filled them with EOD values from
    # the previous trading day.
    dfr_pivot.fillna(method='ffill', inplace=True)

    # Add additional calculated columns to facilitate statistic calculations for each stock.
    dfr_pivot['YearCount'] = dfr_pivot.count(axis=1, numeric_only=True)
    dfr_pivot['UpCount'] = dfr_pivot[dfr_pivot.iloc[:,1:len(dfr_pivot.columns)] > 0].count(axis=1)-1
    dfr_pivot['DownCount'] = dfr_pivot[dfr_pivot.iloc[:,1:len(dfr_pivot.columns)] < 0].count(axis=1)
    dfr_pivot['PctUp'] = dfr_pivot['UpCount']/dfr_pivot['YearCount']
    dfr_pivot['PctDown'] = dfr_pivot['DownCount']/dfr_pivot['YearCount']
    dfr_pivot['AvgReturn'] = dfr_pivot.iloc[:,1:len(dfr_pivot.columns)-5].mean(axis=1)
    dfr_pivot['StDevReturns'] = dfr_pivot.iloc[:,1:len(dfr_pivot.columns)-6].std(axis=1)
    dfr_pivot['67PctDownside'] = dfr_pivot['AvgReturn']-dfr_pivot['StDevReturns']
    dfr_pivot['MaxReturn'] = dfr_pivot.iloc[:,1:len(dfr_pivot.columns)-8].max(axis=1)
    dfr_pivot['MinReturn'] = dfr_pivot.iloc[:,1:len(dfr_pivot.columns)-9].min(axis=1)

    # Add a fictional date column in Python date/time format so the table can be sorted by date. Then sort by Date.
    dfr_pivot['Date'] = '2000-' + dfr_pivot['M-D'].astype(str)
    dfr_pivot['Date'] = pd.to_datetime(dfr_pivot['Date'], infer_datetime_format=True)
    dfr_pivot.sort_values(by='Date',ascending=True, inplace=True)
    
    # Reset the index and round the float values to 4 decimals.
    dfr_pivot.reset_index(inplace=True)
    dfr_pivot = dfr_pivot.round(4)
    
    # Export the final pivot table to CSV for further research later.
    dfr_pivot.to_csv(path_or_buf = my_path + "/data/" + my_ticker + "_dfr_pivot_3mo.csv", index=False)

    # Calculate the trading statistics for the rolling holding periods for the stock.
    pct_uprows = (dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, 'PctUp'].count() / dfr_pivot.loc[:, 'PctUp'].count()).astype(float).round(4)
    max_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, 'MaxReturn'].max()
    min_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, 'MinReturn'].min()
    avg_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > 0, 'AvgReturn'].mean()
    avg_up_return = np.float64(avg_up_return).round(4)
    stdev_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > 0, 'StDevReturns'].mean()
    stdev_up_return = np.float64(stdev_up_return).round(4)
    pct_downside = (avg_up_return - stdev_up_return)
    pct_downside = np.float64(pct_downside).round(4)
    least_pain_pt = dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, '67PctDownside'].max()
    total_years = dfr_pivot['YearCount'].max()
    
    n_consec = 0
    max_n_consec = 0

    for x in dfr_pivot['PctUp']:
        if (x > threshold):
            n_consec += 1
        else: # check for new max, then start again from 1
            max_n_consec = max(n_consec, max_n_consec)
            n_consec = 1

    max_consec_beat = max_n_consec

    try:
        best_sell_date = dfr_pivot.loc[dfr_pivot['67PctDownside'] == least_pain_pt, 'M-D'].iloc[0]
    except:
        best_sell_date = "nan"

    try:
        row = dfr_pivot.loc[dfr_pivot['M-D'] == best_sell_date, 'M-D'].index[0]-90
        col = dfr_pivot.columns.get_loc('M-D')
        best_buy_date = dfr_pivot.iloc[row,col]
    except:
        best_buy_date = "nan"

    # Create the array of stat values and append it to the recommended trade list.
    statsdata = np.array([my_ticker, hold_per, pct_uprows, max_up_return, min_up_return, avg_up_return, stdev_up_return, pct_downside, least_pain_pt, total_years, max_consec_beat, best_buy_date, best_sell_date])
    
    df_tradelist = df_tradelist.append(dict(zip(df_tradelist.columns, statsdata)), ignore_index=True)


In [7]:
# This module grabs each ticker file, transforms it and calculates the statistics needed for a 60 day holding period.

def calc_2month_returns():
    
    global df_tradelist
    global threshold
    hold_per="2 Mos"

    # Convert prices to 3 month returns based on 20 trading days per month.
    dfr = df.pct_change(periods=40)
    dfr.reset_index(level=0, inplace=True)
    dfr.rename(columns={'Close':'Returns'}, inplace=True)
    dfr = dfr.round(4)
    
    # Separate out the date column into separate month, year and day values.
    dfr['Month'] = pd.DatetimeIndex(dfr['Date']).month
    dfr['Day'] = pd.DatetimeIndex(dfr['Date']).day
    dfr['Year'] = pd.DatetimeIndex(dfr['Date']).year
    dfr['M-D'] = dfr['Month'].astype(str)+'-'+dfr['Day'].astype(str)
    pd.set_option('display.max_rows', len(dfr))

    # Pivot the table to show years across the top and Month-Day values in the first column on the left.
    dfr_pivot = dfr.pivot(index='M-D', columns='Year', values='Returns')
    dfr_pivot.reset_index(level=0, inplace=True)
    dfr_pivot = pd.DataFrame(dfr_pivot)
    dfr_pivot.columns.name="Index"

    # The pivot operation created empty cells for weekends and holiday, so I filled them with EOD values from
    # the previous trading day.
    dfr_pivot.fillna(method='ffill', inplace=True)

    # Add additional calculated columns to facilitate statistic calculations for each stock.
    dfr_pivot['YearCount'] = dfr_pivot.count(axis=1, numeric_only=True)
    dfr_pivot['UpCount'] = dfr_pivot[dfr_pivot.iloc[:,1:len(dfr_pivot.columns)] > 0].count(axis=1)-1
    dfr_pivot['DownCount'] = dfr_pivot[dfr_pivot.iloc[:,1:len(dfr_pivot.columns)] < 0].count(axis=1)
    dfr_pivot['PctUp'] = dfr_pivot['UpCount']/dfr_pivot['YearCount']
    dfr_pivot['PctDown'] = dfr_pivot['DownCount']/dfr_pivot['YearCount']
    dfr_pivot['AvgReturn'] = dfr_pivot.iloc[:,1:len(dfr_pivot.columns)-5].mean(axis=1)
    dfr_pivot['StDevReturns'] = dfr_pivot.iloc[:,1:len(dfr_pivot.columns)-6].std(axis=1)
    dfr_pivot['67PctDownside'] = dfr_pivot['AvgReturn']-dfr_pivot['StDevReturns']
    dfr_pivot['MaxReturn'] = dfr_pivot.iloc[:,1:len(dfr_pivot.columns)-8].max(axis=1)
    dfr_pivot['MinReturn'] = dfr_pivot.iloc[:,1:len(dfr_pivot.columns)-9].min(axis=1)

    # Add a fictional date column in Python date/time format so the table can be sorted by date. Then sort by Date.
    dfr_pivot['Date'] = '2000-' + dfr_pivot['M-D'].astype(str)
    dfr_pivot['Date'] = pd.to_datetime(dfr_pivot['Date'], infer_datetime_format=True)
    dfr_pivot.sort_values(by='Date',ascending=True, inplace=True)
    
    # Reset the index and round the float values to 4 decimals.
    dfr_pivot.reset_index(inplace=True)
    dfr_pivot = dfr_pivot.round(4)
    
    # Export the final pivot table to CSV for further research later.
    dfr_pivot.to_csv(path_or_buf = my_path + "/data/" + my_ticker + "_dfr_pivot_2mo.csv", index=False)

    # Calculate the trading statistics for the rolling holding periods for the stock.
    pct_uprows = (dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, 'PctUp'].count() / dfr_pivot.loc[:, 'PctUp'].count()).astype(float).round(4)
    max_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, 'MaxReturn'].max()
    min_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, 'MinReturn'].min()
    avg_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > 0, 'AvgReturn'].mean()
    avg_up_return = np.float64(avg_up_return).round(4)
    stdev_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > 0, 'StDevReturns'].mean()
    stdev_up_return = np.float64(stdev_up_return).round(4)
    pct_downside = (avg_up_return - stdev_up_return)
    pct_downside = np.float64(pct_downside).round(4)
    least_pain_pt = dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, '67PctDownside'].max()
    total_years = dfr_pivot['YearCount'].max()
    
    n_consec = 0
    max_n_consec = 0

    for x in dfr_pivot['PctUp']:
        if (x > threshold):
            n_consec += 1
        else: # check for new max, then start again from 1
            max_n_consec = max(n_consec, max_n_consec)
            n_consec = 1

    max_consec_beat = max_n_consec

    try:
        best_sell_date = dfr_pivot.loc[dfr_pivot['67PctDownside'] == least_pain_pt, 'M-D'].iloc[0]
    except:
        best_sell_date = "nan"

    try:
        row = dfr_pivot.loc[dfr_pivot['M-D'] == best_sell_date, 'M-D'].index[0]-60
        col = dfr_pivot.columns.get_loc('M-D')
        best_buy_date = dfr_pivot.iloc[row,col]
    except:
        best_buy_date = "nan"

    # Create the array of stat values and append it to the recommended trade list.
    statsdata = np.array([my_ticker, hold_per, pct_uprows, max_up_return, min_up_return, avg_up_return, stdev_up_return, pct_downside, least_pain_pt, total_years, max_consec_beat, best_buy_date, best_sell_date])
    
    df_tradelist = df_tradelist.append(dict(zip(df_tradelist.columns, statsdata)), ignore_index=True)


In [8]:
# This module grabs each ticker file, transforms it and calculates the statistics needed for a 30 day holding period.

def calc_1month_returns():
    
    global df_tradelist
    global threshold
    hold_per="1 Mo"

    # Convert prices to 3 month returns based on 20 trading days per month.
    dfr = df.pct_change(periods=20)
    dfr.reset_index(level=0, inplace=True)
    dfr.rename(columns={'Close':'Returns'}, inplace=True)
    dfr = dfr.round(4)
    
    # Separate out the date column into separate month, year and day values.
    dfr['Month'] = pd.DatetimeIndex(dfr['Date']).month
    dfr['Day'] = pd.DatetimeIndex(dfr['Date']).day
    dfr['Year'] = pd.DatetimeIndex(dfr['Date']).year
    dfr['M-D'] = dfr['Month'].astype(str)+'-'+dfr['Day'].astype(str)

    # Pivot the table to show years across the top and Month-Day values in the first column on the left.
    dfr_pivot = dfr.pivot(index='M-D', columns='Year', values='Returns')
    dfr_pivot.reset_index(level=0, inplace=True)
    dfr_pivot = pd.DataFrame(dfr_pivot)
    dfr_pivot.columns.name="Index"

    # The pivot operation created empty cells for weekends and holiday, so I filled them with EOD values from
    # the previous trading day.
    dfr_pivot.fillna(method='ffill', inplace=True)

    # Add additional calculated columns to facilitate statistic calculations for each stock.
    dfr_pivot['YearCount'] = dfr_pivot.count(axis=1, numeric_only=True)
    dfr_pivot['UpCount'] = dfr_pivot[dfr_pivot.iloc[:,1:len(dfr_pivot.columns)] > 0].count(axis=1)-1
    dfr_pivot['DownCount'] = dfr_pivot[dfr_pivot.iloc[:,1:len(dfr_pivot.columns)] < 0].count(axis=1)
    dfr_pivot['PctUp'] = dfr_pivot['UpCount']/dfr_pivot['YearCount']
    dfr_pivot['PctDown'] = dfr_pivot['DownCount']/dfr_pivot['YearCount']
    dfr_pivot['AvgReturn'] = dfr_pivot.iloc[:,1:len(dfr_pivot.columns)-5].mean(axis=1)
    dfr_pivot['StDevReturns'] = dfr_pivot.iloc[:,1:len(dfr_pivot.columns)-6].std(axis=1)
    dfr_pivot['67PctDownside'] = dfr_pivot['AvgReturn']-dfr_pivot['StDevReturns']
    dfr_pivot['MaxReturn'] = dfr_pivot.iloc[:,1:len(dfr_pivot.columns)-8].max(axis=1)
    dfr_pivot['MinReturn'] = dfr_pivot.iloc[:,1:len(dfr_pivot.columns)-9].min(axis=1)

    # Add a fictional date column in Python date/time format so the table can be sorted by date. Then sort by Date.
    dfr_pivot['Date'] = '2000-' + dfr_pivot['M-D'].astype(str)
    dfr_pivot['Date'] = pd.to_datetime(dfr_pivot['Date'], infer_datetime_format=True)
    dfr_pivot.sort_values(by='Date',ascending=True, inplace=True)
    
    # Reset the index and round the float values to 4 decimals.
    dfr_pivot.reset_index(inplace=True)
    dfr_pivot = dfr_pivot.round(4)
    
    # Export the final pivot table to CSV for further research later.
    dfr_pivot.to_csv(path_or_buf = my_path + "/data/" + my_ticker + "_dfr_pivot_1mo.csv", index=False)

    # Calculate the trading statistics for the rolling holding periods for the stock.
    pct_uprows = (dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, 'PctUp'].count() / dfr_pivot.loc[:, 'PctUp'].count()).astype(float).round(4)
    max_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, 'MaxReturn'].max()
    min_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, 'MinReturn'].min()
    avg_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > 0, 'AvgReturn'].mean()
    avg_up_return = np.float64(avg_up_return).round(4)
    stdev_up_return = dfr_pivot.loc[dfr_pivot['PctUp'] > 0, 'StDevReturns'].mean()
    stdev_up_return = np.float64(stdev_up_return).round(4)
    pct_downside = (avg_up_return - stdev_up_return)
    pct_downside = np.float64(pct_downside).round(4)
    least_pain_pt = dfr_pivot.loc[dfr_pivot['PctUp'] > threshold, '67PctDownside'].max()
    total_years = dfr_pivot['YearCount'].max()

    n_consec = 0
    max_n_consec = 0

    for x in dfr_pivot['PctUp']:
        if (x > threshold):
            n_consec += 1
        else: # check for new max, then start again from 1
            max_n_consec = max(n_consec, max_n_consec)
            n_consec = 1

    max_consec_beat = max_n_consec

    try:
        best_sell_date = dfr_pivot.loc[dfr_pivot['67PctDownside'] == least_pain_pt, 'M-D'].iloc[0]
    except:
        best_sell_date = "nan"

    try:
        row = dfr_pivot.loc[dfr_pivot['M-D'] == best_sell_date, 'M-D'].index[0]-30
        col = dfr_pivot.columns.get_loc('M-D')
        best_buy_date = dfr_pivot.iloc[row,col]
    except:
        best_buy_date = "nan"

    # Create the array of stat values and append it to the recommended trade list.
    statsdata = np.array([my_ticker, hold_per, pct_uprows, max_up_return, min_up_return, avg_up_return, stdev_up_return, pct_downside, least_pain_pt, total_years, max_consec_beat, best_buy_date, best_sell_date])
       
    df_tradelist = df_tradelist.append(dict(zip(df_tradelist.columns, statsdata)), ignore_index=True)

In [9]:

# Read CSV files by ticker, transform and extract stats from each one.

for index, ticker in df_sp500_tickers.iterrows():
    my_ticker = ticker['Symbol']

    df = pd.read_csv (my_path + "/data/" + my_ticker + ".csv")
    df.set_index('Date', inplace=True)
    df = df['Close']
    df = pd.DataFrame(df, columns=['Close'])
    
    calc_1month_returns()
    
    calc_2month_returns()
    
    calc_3month_returns()




/Users/bnsheehy/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in long_scalars
/Users/bnsheehy/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in long_scalars


In [10]:
# Preview the end of the trade list to see how many rows and make sure the data is showing up where it is supposed to.

df_tradelist.tail(10)

,my_ticker,hold_per,pct_uprows,max_up_return,min_up_return,avg_up_return,stdev_up_return,pct_downside,least_pain_pt,total_years,max_consec_beat,best_buy_date,best_sell_date
1499,YUM,3 Mos,0.1377,0.5091,-0.2379,0.0344,0.1169,-0.0825,0.0246,24,39,2-6,5-6
1500,ZBH,1 Mo,0.0303,0.177,-0.1373,0.0114,0.065,-0.0536,-0.0048,20,5,12-16,1-17
1501,ZBH,2 Mos,0.0771,0.297,-0.1404,0.0238,0.0915,-0.0677,0.0092,20,7,12-21,2-21
1502,ZBH,3 Mos,0.0964,0.3439,-0.2127,0.0393,0.1146,-0.0753,0.0451,20,24,11-21,2-21
1503,ZION,1 Mo,0.0,nan,nan,0.0145,0.0917,-0.0772,nan,41,1,nan,nan
1504,ZION,2 Mos,0.0028,0.356,-0.3402,0.0274,0.1285,-0.1011,-0.0786,41,2,1-17,3-17
1505,ZION,3 Mos,0.0,nan,nan,0.0452,0.1586,-0.1134,nan,41,1,nan,nan
1506,ZTS,1 Mo,0.1295,0.1859,-0.0897,0.028,0.0588,-0.0308,0.0126,8,8,5-11,6-10
1507,ZTS,2 Mos,0.2314,0.2091,-0.1829,0.0486,0.0799,-0.0313,0.0285,8,24,10-30,12-30
1508,ZTS,3 Mos,0.4793,0.243,-0.1952,0.0552,0.0817,-0.0265,0.0355,8,39,2-19,5-19


In [11]:
# Convert the trade list to a Pandas dataframe.
df_tradelist = pd.DataFrame(df_tradelist)

In [12]:
# Clean it up by removing rows with NaN's and infinity values.
df_tradelist.replace("inf", np.nan, inplace=True)
df_tradelist.dropna(inplace=True)
df_tradelist = df_tradelist[~df_tradelist.max_up_return.str.contains("nan")]

In [13]:
# Create a final filtered and sorted recommended trade list.
# I filtered out the stocks where the percentage of 80%+ up years was less than 1 out of 10. In other words,
# with about 250 trading days in the year, each stock on the list needs to show at least 25 days where the holding
# period ending on that day showed a gain for at least 4 out of 5 years surveyed.
# Then I added a dummy date column, sorted by date, and the dropped the dummy column.

df_tradelist['pct_uprows'] = df_tradelist['pct_uprows'].astype('Float64')
df_tradelist_fs = df_tradelist[(df_tradelist['pct_uprows'] > 0.1)].copy()


df_tradelist_fs['Date'] = '2000-' + df_tradelist_fs['best_buy_date'].astype(str)
df_tradelist_fs['Date'] = pd.to_datetime(df_tradelist_fs['Date'], infer_datetime_format=True)
df_tradelist_fs.sort_values(by='Date',ascending=True, inplace=True)
df_tradelist_fs.drop('Date', axis=1, inplace=True)
df_tradelist_fs.drop_duplicates(subset="my_ticker", keep='first', inplace=True)


In [14]:
# Preview the results. There should be between 50 and 100 stock trades on the final list.
df_tradelist_fs.head()

,my_ticker,hold_per,pct_uprows,max_up_return,min_up_return,avg_up_return,stdev_up_return,pct_downside,least_pain_pt,total_years,max_consec_beat,best_buy_date,best_sell_date
653,HCA,3 Mos,0.2452,0.396,-0.2151,0.0755,0.1393,-0.0638,0.0614,10,29,1-2,4-1
1172,QRVO,3 Mos,0.3278,0.4512,-0.3229,0.1031,0.2309,-0.1278,0.1053,6,63,1-2,4-1
668,HII,3 Mos,0.3664,0.3715,-0.3056,0.0892,0.1357,-0.0465,0.1076,10,63,1-2,4-1
971,MSCI,3 Mos,0.2424,0.5458,-0.2672,0.067,0.14,-0.073,0.0405,13,22,1-4,4-3
590,FTNT,3 Mos,0.3857,0.5178,-0.3961,0.1106,0.2099,-0.0993,0.1261,11,71,1-4,4-3


In [15]:
# Export the original and the filtered trade lists to CSV files for execution and/or further research if desired.
df_tradelist.to_csv(path_or_buf = my_path + "/df_tradelist.csv", index=False)
df_tradelist_fs.to_csv(path_or_buf = my_path + "/df_tradelist_fs.csv", index=False)